In [51]:
import os, shutil
import fileinput as fi
import glob
import moviepy.editor as mp
import pandas as pd
import random
import time



os.chdir(path=r"D:\02_batch_2")
print("Current Working Directory " , os.getcwd())


Current Working Directory  D:\02_batch_2


### Arabic Annotation collection

In [56]:
df_dirname = []
for folder in sorted(os.listdir(os.getcwd())):
    df_dirname.append((folder))

df_dirname = pd.DataFrame(df_dirname, columns=['Folder'])
df_dirname.sort_values(by='Folder',inplace=True)
df_dirname

,Folder
0,37_degrees_series
1,Almajd_kids
2,Ba3d_altes3een_Podcast
3,C4N
4,Dupamicaffeine
5,Mojtm3_Ethraa
6,Naif_aljaweni
7,Sando9_aswad_turki_alfaisal
8,Shabab_Ela3mal
9,Update_NewMedia_Podcast


li = []

try:

    for i,dname in enumerate(df_dirname['Folder']):
        path = r'D:/02_batch_2/'
        files = glob.glob(path + dname + '/*.txt')
        print(f'Successfully created dataframe for {dname}')
        for i,f in enumerate(files):

            parse_error = False
            file_not_found = False

            stock = os.path.basename(f)            
            temp_df = pd.read_csv(f, delimiter="\t", error_bad_lines=False)
            temp_df['filePath'] = path + dname + '/'
            temp_df['textFileName'] = stock     
            temp_df['audioFileName'] = temp_df['textFileName'].replace('.txt', '.wav', regex=True)
            temp_df['duration'] = temp_df.stop-temp_df.start
            li.append(temp_df)
            print(f'Successfully created dataframe for {stock} with shape {temp_df.shape}')

except pd.errors.ParserError:
    parse_error = True
    temp_df = pd.read_csv(f, delimiter="\t",skiprows=4)
    li.append(temp_df)
    print(f'Error for {dname} {stock} with shape {temp_df.shape}')

else:
    li.append(temp_df)


df = pd.concat(li, axis=0)
print(df.shape)
df.head(3)

In [53]:
li = []
path = r'D:/02_batch_2/'

for i,dname in enumerate(df_dirname['Folder']):

    files = glob.glob(path + dname + '/*.txt')
    #print(f'Successfully created dataframe for {dname}')
    for i,f in enumerate(files):

        parse_error = False
        file_not_found = False

        stock = os.path.basename(f)            
        temp_df = pd.read_csv(f, delimiter="\t", error_bad_lines=False)
        temp_df['filePath'] = path + dname + '/'
        temp_df['textFileName'] = stock     
        temp_df['audioFileName'] = temp_df['textFileName'].replace('.txt', '.wav', regex=True)
        temp_df['duration'] = temp_df.stop-temp_df.start
        li.append(temp_df)
      #  print(f'Successfully created dataframe for {stock} with shape {temp_df.shape}')

df = pd.concat(li, axis=0)
print(df.shape)
df.head(2)

b'Skipping line 18: expected 5 fields, saw 7\n'
b'Skipping line 12: expected 5 fields, saw 6\n'
b'Skipping line 5: expected 5 fields, saw 8\n'


(507921, 9)


,speaker,gender,start,stop,transcript,filePath,textFileName,audioFileName,duration
0,speakeA,M,8.998978,23.522329,الحقوني الحقوني بطني بطني بموت ما أقدر الحقون...,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-0.txt,37_degrees_series_0-0.wav,14.523351
1,speakeA,M,23.522329,25.772074,أضحك عليكو بس أختبركو.,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-0.txt,37_degrees_series_0-0.wav,2.249745


In [54]:
df_percent = df.groupby(['filePath','textFileName']).agg({'duration':['sum']}).reset_index()
df_percent=df_percent.droplevel(axis=1,level=1)
df_percent.rename(columns={'duration':'duration_sum'},inplace=True)

def as_perc(value, total):
    return value/float(total)

res = pd.DataFrame(df_percent.duration_sum.apply(as_perc, total=df_percent.duration_sum.sum()))
df_percent['percent'] = res

df_percent['req_duration'] = round(df_percent['percent']*180000)
df_percent.head(5)

,filePath,textFileName,duration_sum,percent,req_duration
0,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-0.txt,161.231694,0.000039,7.0
1,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-1.txt,227.017000,0.000054,10.0
2,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-2.txt,100.843458,0.000024,4.0
3,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-3.txt,112.098840,0.000027,5.0
4,D:/02_batch_2/37_degrees_series/,37_degrees_series_0-4.txt,138.146208,0.000033,6.0


In [55]:
df_req_percent = df_percent.groupby('filePath').agg({'req_duration':'sum','percent':'sum'}).reset_index()
df_req_percent.head(3)

,filePath,req_duration,percent
0,D:/02_batch_2/37_degrees_series/,1119.0,0.006219
1,D:/02_batch_2/Almajd_kids/,4495.0,0.024986
2,D:/02_batch_2/Ba3d_altes3een_Podcast/,2741.0,0.015221


In [ ]:
i = 0
x = 0

df_selection = pd.DataFrame(columns=['src_filepath','src_filepath_wav','duration']) 

for i,fpath in enumerate(df_req_percent['filePath']):
    sum_duration = 0.0
    req_duration = df_req_percent.loc[i,'req_duration']
    filter_fpath = df_percent[df_percent['filePath'] == fpath]    

    for ifpath in filter_fpath['textFileName']:
        if (req_duration >= sum_duration):
            n = random.randint(0,len(filter_fpath)-1)    
                
            df_selection.loc[x,'src_filepath'] = str(filter_fpath.iloc[n,0]) + str(filter_fpath.iloc[n,1])
          #  df_selection.loc[x,'src_filepath_txt'] = str(filter_fpath.iloc[n,1])
            df_selection.loc[x,'src_filepath_wav'] = df_selection.loc[x,'src_filepath'].replace('.txt', '.wav')
            df_selection.loc[x,'duration'] = filter_fpath.iloc[n,2].astype('float32')

            sum_duration = sum_duration + filter_fpath.iloc[n,2].astype('float32')
            shutil.copy(df_selection.loc[x,'src_filepath'], 'D:/nf/audio/')
            shutil.copy(df_selection.loc[x,'src_filepath_wav'], 'D:/nf/audio/')

            x+=1           


df_selection.head(1000)
        


,src_filepath,src_filepath_wav,duration
0,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,211.509109
1,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,176.867233
2,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,161.231689
3,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,159.796692
4,D:/02_batch_2/37_degrees_series/37_degrees_ser...,D:/02_batch_2/37_degrees_series/37_degrees_ser...,280.323547
...,...,...,...
782,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,209.83905
783,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,192.945999
784,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,236.317001
785,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,D:/02_batch_2/zainab_almatroodi/zainab_almatro...,281.45401


# AUXILLIARY CODE

### Video Clipping

In [ ]:
hours = 180000;
duration_sum = 0;
x=0;
dest_filepath = r'D:/nf/audio/'

df_selection = pd.DataFrame(
    columns=['row_no','src_filepath','src_filepath_txt','src_filepath_wav','dest_filepath','dest_filepath_txt','dest_filepath_wav','duration','start','end']
    )

while (hours>=duration_sum):
    
    n = random.randint(0,len(df))
    fname = str(df.iloc[n,5] + df.iloc[n,7])
    duration_sum = duration_sum + df.iloc[n,8].astype('float32')

    df_selection[x,0] = n
    df_selection[x,1] = str(df.iloc[n,5])
    df_selection[x,2] = str(df.iloc[n,6])
    df_selection[x,3] = str(df.iloc[n,7])
    df_selection[x,4] = dest_filepath
    df_selection[x,5] = str(x) + "_" + str(df.iloc[n,6])
    df_selection[x,6] = str(x) + "_" + str(df.iloc[n,7])
    df_selection[x,7] = df.iloc[n,8].astype('float32')
    df_selection[x,8] = df.iloc[n,2].astype('float32')
    df_selection[x,9] = df.iloc[n,3].astype('float32')

    x=x+1  
    start = time.time()
    my_clip = mp.AudioFileClip(fname)    
    
    clip = my_clip.subclip(df.iloc[n,2],df.iloc[n,3])
    clip.write_audiofile(dest_filepath + str(x) + "_"+ str(df.iloc[n,7]) +".mp3")
    #time.sleep(1)
    end = time.time()
    

    print(duration_sum)
    print(f"Runtime of the program is {end - start}")
    # clip.write_audiofile(dest_filepath + df_selection[x,6])


df_selection.head(4) 

### Directory Structure

In [ ]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))



In [ ]:
list_files(r"D:\02_batch_2")